In [ ]:
from tensorflow.python.client import  device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8128387713982759320
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14444920832
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6189629949365105509
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn import svm

from sklearn import metrics 
from sklearn.metrics import classification_report

from tensorflow.keras import layers, models
from keras.layers import concatenate
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import h5py
 
DATA_DIR = "/content/drive/MyDrive/Radar/Radar_600.mat"
 
data = h5py.File(DATA_DIR,'r')
 
data.keys()

<KeysViewHDF5 ['Data', 'Labels']>

In [ ]:
import numpy as np
 
X = np.transpose(np.array(data['Data']))
Y = np.reshape(np.array(data['Labels']),[-1,])
X = np.reshape(X,[len(X),3, 1024,1])
 
print(X.shape)
print(Y.shape)
Y = Y - 1
Y = Y.astype(np.int32)
Y.dtype

(5400, 3, 1024, 1)
(5400,)


dtype('int32')

In [ ]:
input_shape = (1024,1)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
#from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model
 
# define the model
def define_model(input_shape, num_channels):
    flat_layers = []
    input_layers = []
    ksize = 7
    psize = 5
    for i in range(num_channels):
      inputs = layers.Input(shape=(input_shape))
      #x = layers.BatchNormalization()(inputs)
 
      x = layers.Conv1D(filters=32,kernel_size=ksize, activation='relu')(inputs)
      x = layers.MaxPooling1D(psize)(x)
      x = layers.Conv1D(filters=48,kernel_size=ksize, activation='relu')(x)
      x = layers.Conv1D(filters=64,kernel_size=ksize, activation='relu')(x)
      x = layers.MaxPooling1D(psize)(x)
      x = layers.Conv1D(filters=32,kernel_size=ksize, activation='relu')(x)
      x = layers.Conv1D(filters=128,kernel_size=ksize, activation='relu')(x)
      x = layers.MaxPooling1D(psize)(x)

      #x = layers.Dropout(0.1)(x)
      flat = layers.Flatten()(x)
      #flat = layers.Dense(256, activation='relu')(x)
 
      flat_layers.append(flat)
      input_layers.append(inputs)
   
    # merge
    merged = concatenate(flat_layers)
    # interpretation
    output = layers.Dense(9, activation='softmax')(merged)
    model = models.Model(inputs=input_layers, outputs=output)
    # compile
    model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy,
                  optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    # summarize
    #print(model.summary())
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [ ]:
from sklearn.model_selection import KFold
train_acc_per_fold = []
test_acc_per_fold = []
kfold = KFold(n_splits = 10, shuffle=True)
fold_no = 1
while fold_no <=10:
  X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.1)
  X_tes = []
  for i in range(3):
    X_tes.append(X_test[:,i,:,:])
  for train, test in kfold.split(X, Y):
    model = define_model(input_shape, 3)
    X_tr = []
    X_te = []
    for i in range(3):
      X_tr.append(X[train][:,i,:,:])
      X_te.append(X[test][:,i,:,:])
  
    history = model.fit(X_tr, Y[train], epochs =50, batch_size=128, shuffle = True,validation_data = (X_te, Y[test]))
    test_loss, test_acc = model.evaluate(X_tes,  Y_test, verbose=2)
    print("test accuracy in fold {} : {} %".format(fold_no, test_acc*100))
    fold_no = fold_no + 1
    test_acc_per_fold.append(test_acc*100)
    
print(f'> Overall Test Accuracy: {np.mean(test_acc_per_fold)} (+- {np.std(test_acc_per_fold)})')

Epoch 1/50
35/35 [==============================] - 10s 85ms/step - loss: 2.1526 - accuracy: 0.1445 - val_loss: 2.0569 - val_accuracy: 0.2572
Epoch 2/50
35/35 [==============================] - 1s 23ms/step - loss: 1.9131 - accuracy: 0.2851 - val_loss: 1.8988 - val_accuracy: 0.2510
Epoch 3/50
35/35 [==============================] - 1s 24ms/step - loss: 1.6545 - accuracy: 0.3779 - val_loss: 1.4710 - val_accuracy: 0.4403
Epoch 4/50
35/35 [==============================] - 1s 23ms/step - loss: 1.3861 - accuracy: 0.4710 - val_loss: 1.3171 - val_accuracy: 0.5123
Epoch 5/50
35/35 [==============================] - 1s 24ms/step - loss: 1.1724 - accuracy: 0.5439 - val_loss: 1.2118 - val_accuracy: 0.5391
Epoch 6/50
35/35 [==============================] - 1s 25ms/step - loss: 1.0623 - accuracy: 0.5880 - val_loss: 1.0518 - val_accuracy: 0.5700
Epoch 7/50
35/35 [==============================] - 1s 24ms/step - loss: 0.8967 - accuracy: 0.6511 - val_loss: 0.9226 - val_accuracy: 0.6440
Epoch 8/50
3